In [72]:
# https://github.com/derekgreene/topic-model-tutorial/blob/master/1%20-%20Text%20Preprocessing.ipynb
from pathlib import Path
import operator, joblib
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [73]:
in_path = Path("data") / "articles.txt"
raw_documents = []
snippets = []
with open(in_path, "r", encoding="utf8") as fin:
    for line in fin.readlines():
        text = line.strip()
        raw_documents.append( text )
        # keep a short snippet of up to 100 characters as a title for each article
        snippets.append(text[0:min(len(text),100)])
print("Read %d raw text documents" % len(raw_documents))

Read 28 raw text documents


In [74]:
custom_stop_words = []
with open( "stopwords.txt", "r", encoding="utf8") as fin:
    for line in fin.readlines():
        custom_stop_words.append(line.strip())
# note that we need to make it hashable
print("Stopword list has %d entries" % len(custom_stop_words))

Stopword list has 1298 entries


In [75]:
# use a custom stopwords list, set the minimum term-document frequency to 20
vectorizer = CountVectorizer(stop_words = custom_stop_words, min_df=5)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d document-term matrix" % (A.shape[0], A.shape[1]))

Created 28 X 78 document-term matrix


In [76]:
terms =  list(vectorizer.get_feature_names_out())
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 78 distinct terms


In [77]:
joblib.dump((A,terms,snippets), "articles-raw.pkl") 

['articles-raw.pkl']

In [78]:
# we can pass in the same preprocessing parameters
vectorizer = TfidfVectorizer(stop_words=custom_stop_words, min_df = 5)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]) )

Created 28 X 78 TF-IDF-normalized document-term matrix


In [79]:
terms =  list(vectorizer.get_feature_names_out())
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 78 distinct terms


In [80]:
def rank_terms(A, term):
    # get the sums over each column
    sums = A.sum(axis=0)
    # map weights to the terms
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    # rank the terms by their weight over all documents
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

In [81]:
ranking = rank_terms(A, terms)
for i, pair in enumerate(ranking[0:100]):
    print( "%02d. %s (%.2f)" % (i+1, pair[0], pair[1] ))

01. systems (4.69)
02. software (3.92)
03. cps (3.18)
04. ecosystems (3.09)
05. data (2.73)
06. cyber (2.67)
07. ecosystem (2.66)
08. physical (2.61)
09. digital (2.54)
10. innovation (2.34)
11. engineering (2.26)
12. computing (1.74)
13. analysis (1.71)
14. challenges (1.68)
15. platform (1.60)
16. based (1.51)
17. development (1.51)
18. smart (1.48)
19. applications (1.44)
20. internet (1.44)
21. cloud (1.43)
22. devices (1.43)
23. study (1.41)
24. model (1.39)
25. models (1.38)
26. time (1.25)
27. literature (1.24)
28. approach (1.22)
29. paper (1.18)
30. services (1.18)
31. systematic (1.14)
32. support (1.12)
33. interoperability (1.11)
34. provide (1.11)
35. industry (1.05)
36. design (1.03)
37. opportunities (0.99)
38. solutions (0.99)
39. complex (0.97)
40. management (0.97)
41. tools (0.97)
42. dynamic (0.94)
43. potential (0.92)
44. platforms (0.92)
45. evolution (0.85)
46. architecture (0.85)
47. level (0.85)
48. future (0.85)
49. increasingly (0.82)
50. process (0.77)
51. b

In [82]:
joblib.dump((A,terms,snippets), "articles-tfidf.pkl") 

['articles-tfidf.pkl']